In [25]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
import datetime

# データ生成用モジュール
# クラスと関数だけ集めておくことにする


In [26]:

# ユーティリティ
# 全角→半角
def get_hankaku_from_zenkaku(text):
    return text.translate(str.maketrans({chr(0xFF01 + i): chr(0x21 + i) for i in range(94)}))

# 日付str → datetime
def get_formatted_date(datestr):
    return datetime.datetime.strptime(datestr,"%Y/%m/%d")


In [27]:


#データ整形

# ジャニーズ

class JanniesScraper:                
    
    def __init__(self, datestr, soup_url ,artname):
        self.df = pd.DataFrame()
        self.soup =  BeautifulSoup(open(soup_url),"html.parser")
        self.datestr = datestr
        self.artistname = artname
        
#   アーティスト名(トリートメントグループにするかどうかで必要)
    def add_artistname(self):
        self.df["artistname"] = str(self.artistname)
        
        
#         年齢(グループ平均年齢)
    def add_age(self):
        if self.artistname =="ジャニーズJr":
            self.df["age"] = 22.3
        elif self.artistname == "セクシーゾーン":
            self.df["age"] = 22.2
        elif self.artistname == "キスマイフットツー":
            self.df["age"]= 30.5
        elif self.artistname == "NEWS":
            self.df["age"] = 32.75
        else:
            self.df["age"] = 25
        
    # 公演名
    # str
    def add_name(self):
        kouen_name_list= list(map(lambda x:x.string.strip().replace("\n",""),self.soup.select(".list-ticket-name>p")))
        self.df["name"] = kouen_name_list 
         

    #公演日時
    # Timestamp YYYY/mm/dd
    def add_date(self):
        p = re.compile(r"<[^>]*?>")
        date_list = list( map(\
                             lambda x:
                                    datetime.datetime.strptime( #str→datetime
                                        "20"+ #元の文字が19/05/17で年数に不備が生じるので、先頭に20をつける
                                        p.sub("",str(x)) #タグ除去
                                        .replace(" ","") #空白除去
                                        .replace("\n", "")[:8],"%Y/%m/%d"),#改行除去、曜日を削除、フォーマット変換
                             self.soup.select(".list-ticket-date>span")))
        # type(kouen_data_list[0])
        self.df["date"] = date_list
    
#  　フォーマットユーティリティ
    def format_to_slash(self):
        return self.datestr.replace("-", "/")
    
    #チケットデータ収集日  timestamp YYYY/mm/dd
    def add_collected_at(self):
        slash_date = self.format_to_slash()
        self.df["collected_at"] = datetime.datetime.strptime(slash_date,"%Y/%m/%d")
   
    #公演までの残り日数 int
    def add_left_days(self):
        self.df["left_days"] = (self.df["date"] -self.df["collected_at"]).dt.days  #日時差をint変換
    
    #  ジャニーズダミー 0 1
    def add_is_jannies(self):
        IS_JANNIES = 1
        self.df["is_jannies"] =IS_JANNIES

        
    # 購入可能ダミー　買えるなら1, 買えないなら0　買えるとき →　.list-ticket-order内部にformがあるとき
    def add_available(self):
        list_ticket_order = self.soup.select(".list-ticket-order")
        available = []
        for i, order in enumerate(list_ticket_order):
            if(order.find("form")):
                available.append(1)
            else:
                available.append(0)
        self.df["available"] = available
    

        # 取引あたり価格  int 
    def add_deal_price_list(self):
        deal_price_list = list(map(lambda x:
                          int(re.findall(r"￥\d+,\d+",
                                     str(x))[0][1:]
                                     .replace(",", "")),
                          self.soup.select(".list-ticket-price")))
        self.df["price_per_deal"] = deal_price_list
        
    
    #連番数 int
    def add_unit(self):
        price_list = self.soup.select(".list-ticket-price")
        unit_list = []
        for prc in price_list:
            if "ticket-sellunit" in str(prc):
                unit_list.append(
                    int(prc.select(".ticket-sellunit")[0].get_text()))
            else:
                unit_list.append(1)
        self.df["unit"] = unit_list

#   単位あたり価格 intのつもり
    def add_price_per_unit(self):
        self.df["price_per_unit"] = self.df["price_per_deal"]/ self.df["unit"]

#     取引あたり価格/連番/単位あたり価格の順で ただのファサード
    def add_price_info_bundle(self):
        self.add_deal_price_list()
        self.add_unit() 
        self.add_price_per_unit()
    
    #　チケット転売施行ダミー 01
    # 収集日によって決定　施行後なら1、施行前なら0
    def add_after(self):
        ENFORCE_DATE = "2019-06-14"
        self.df["after"]=   (self.df["collected_at"] >=ENFORCE_DATE)*1
    
        
#  施行前後ダミー×アーティストグループダミーの交差項
    
    def add_cross_term(self):
        self.df["after_and_js_jannies"] = self.df["after"] * self.df["is_jannies"]
        

#         (メインロジック):列の追加から書き出し
    def store_data(self):
#         最初に公演名をstoreして、長さを確保してからアーティスト名などの代入系を実行
# 　　　　代入系→soup系の順でやると代入系はNaNになる
        self.add_name() #公演名
        self.add_date() #日付
        self.add_collected_at() #収集日
        self.add_left_days() #残り日数
        self.add_is_jannies() #グループダミー
        self.add_available() #購入可能
        self.add_price_info_bundle() #取引価格/枚数/単位価格
        self.add_after() #法施行前後ダミー
        self.add_cross_term() #交差項
        self.add_artistname()#アーティスト名
        self.add_age() #アーティスト年齢
        self.drop_outlier()# 外れ値を落とす
        
    def drop_outlier(self):
        self.df = self.df[self.df["price_per_unit"] >= 1000]

        

        
        #     CSV書き出し
    def write_in_csv(self, filename):
        self.df.to_csv(filename,index=0)
    


In [21]:
# テスト
# TODO ScraperインスタンスにDFとSOUPと日付を保持するよう改修
# コンストラクタにURLと日付

# 生成したCSVをファイル構造を同じくする/CSV/アーティスト名/日付/ticket-data:n.csvに保存
# アーティストごと結合したCSVを /CSV/アーティスト/united-by-artist.csvに保存
# ジャニーズ結合データを/CSV/united-by-jannies.csvに保存
# パスをインスタンス変数にしてwriteも中で行えるようにする

# 演歌グループで同じ構造のクラス生成
# 書き込みまでデータ化
soup_url = "./jannies.html"
sc = JanniesScraper(datestr="2019-05-16",soup_url=soup_url, artname="ジャニーズJr")
sc.store_data()
sc.write_in_csv("./sample_jannies.csv")
df = pd.read_csv("sample_jannies.csv")
df


> <ipython-input-20-1beed2376019>(120)store_data()
    118     def store_data(self):
    119         from IPython.core.debugger import Pdb; Pdb().set_trace()
--> 120         self.add_name() #公演名
    121         self.add_date() #日付
    122         self.add_collected_at() #収集日

ipdb> self.df
Empty DataFrame
Columns: []
Index: []
ipdb> c
> <ipython-input-20-1beed2376019>(130)store_data()
    128         self.add_cross_term() #交差項
    129         from IPython.core.debugger import Pdb; Pdb().set_trace()
--> 130         self.add_artistname()#アーティスト名
    131         from IPython.core.debugger import Pdb; Pdb().set_trace()
    132 

ipdb> self.df
                                      name       date collected_at  left_days  \
0   HiHi Jets ジャニーズ銀座2019 Tokyo Experience 2019-05-17   2019-05-16          1   
1   HiHi Jets ジャニーズ銀座2019 Tokyo Experience 2019-05-17   2019-05-16          1   
2   HiHi Jets ジャニーズ銀座2019 Tokyo Experience 2019-05-17   2019-05-16          1   
3   HiHi Jets ジャニーズ銀座2019 Tok

ipdb> c
--Return--
None
> <ipython-input-20-1beed2376019>(130)store_data()
    128         self.add_cross_term() #交差項
    129         from IPython.core.debugger import Pdb; Pdb().set_trace()
--> 130         self.add_artistname()#アーティスト名
    131         from IPython.core.debugger import Pdb; Pdb().set_trace()
    132 

ipdb> self.df
                                      name       date collected_at  left_days  \
0   HiHi Jets ジャニーズ銀座2019 Tokyo Experience 2019-05-17   2019-05-16          1   
1   HiHi Jets ジャニーズ銀座2019 Tokyo Experience 2019-05-17   2019-05-16          1   
2   HiHi Jets ジャニーズ銀座2019 Tokyo Experience 2019-05-17   2019-05-16          1   
3   HiHi Jets ジャニーズ銀座2019 Tokyo Experience 2019-05-17   2019-05-16          1   
4   HiHi Jets ジャニーズ銀座2019 Tokyo Experience 2019-05-17   2019-05-16          1   
5   HiHi Jets ジャニーズ銀座2019 Tokyo Experience 2019-05-17   2019-05-16          1   
6   HiHi Jets ジャニーズ銀座2019 Tokyo Experience 2019-05-17   2019-05-16          1   
7   HiHi Jets ジャニー

ipdb> c


,name,date,collected_at,left_days,is_jannies,available,price_per_deal,unit,price_per_unit,after,after_and_js_jannies,artistname
0,HiHi Jets ジャニーズ銀座2019 Tokyo Experience,2019-05-17,2019-05-16,1,1,1,77000,1,77000.0,0,0,ジャニーズJr
1,HiHi Jets ジャニーズ銀座2019 Tokyo Experience,2019-05-17,2019-05-16,1,1,0,99000,1,99000.0,0,0,ジャニーズJr
2,HiHi Jets ジャニーズ銀座2019 Tokyo Experience,2019-05-17,2019-05-16,1,1,0,150000,1,150000.0,0,0,ジャニーズJr
3,HiHi Jets ジャニーズ銀座2019 Tokyo Experience,2019-05-17,2019-05-16,1,1,1,215000,1,215000.0,0,0,ジャニーズJr
4,HiHi Jets ジャニーズ銀座2019 Tokyo Experience,2019-05-17,2019-05-16,1,1,1,230000,1,230000.0,0,0,ジャニーズJr
5,HiHi Jets ジャニーズ銀座2019 Tokyo Experience,2019-05-17,2019-05-16,1,1,1,230000,2,115000.0,0,0,ジャニーズJr
6,HiHi Jets ジャニーズ銀座2019 Tokyo Experience,2019-05-17,2019-05-16,1,1,0,230000,1,230000.0,0,0,ジャニーズJr
7,HiHi Jets ジャニーズ銀座2019 Tokyo Experience,2019-05-17,2019-05-16,1,1,0,250000,1,250000.0,0,0,ジャニーズJr
8,HiHi Jets ジャニーズ銀座2019 Tokyo Experience,2019-05-17,2019-05-16,1,1,1,280000,2,140000.0,0,0,ジャニーズJr
9,HiHi Jets ジャニーズ銀座2019 Tokyo Experience,2019-05-18,2019-05-16,2,1,1,58000,2,29000.0,0,0,ジャニーズJr


In [28]:
soup_url = "../細川たかし/2019-09-20/ticket-data1.html"
sc1 = JanniesScraper(datestr="2019-09-20",soup_url=soup_url,artname="細川たかし")
sc1.store_data()
sc1.df["date"] = sc1.df["date"] - datetime.timedelta(days=90)
sc1.df["collected_at"] = sc1.df["collected_at"] - datetime.timedelta(days=90)
sc1.df

,name,date,collected_at,left_days,is_jannies,available,price_per_deal,unit,price_per_unit,after,after_and_js_jannies,artistname,age
0,新・BS日本のうた,2019-07-05,2019-06-22,13,1,1,3200,1,3200.0,1,1,細川たかし,25
1,新・BS日本のうた,2019-07-05,2019-06-22,13,1,1,3500,1,3500.0,1,1,細川たかし,25
2,新・BS日本のうた,2019-07-05,2019-06-22,13,1,0,3000,1,3000.0,1,1,細川たかし,25
3,新・BS日本のうた,2019-07-05,2019-06-22,13,1,1,3300,1,3300.0,1,1,細川たかし,25
4,新・BS日本のうた,2019-07-05,2019-06-22,13,1,1,3980,1,3980.0,1,1,細川たかし,25
5,新・BS日本のうた,2019-07-05,2019-06-22,13,1,0,5000,1,5000.0,1,1,細川たかし,25
6,新・BS日本のうた,2019-07-05,2019-06-22,13,1,0,2800,1,2800.0,1,1,細川たかし,25
7,新・BS日本のうた,2019-07-05,2019-06-22,13,1,0,2800,1,2800.0,1,1,細川たかし,25
8,新・BS日本のうた,2019-07-05,2019-06-22,13,1,0,3000,1,3000.0,1,1,細川たかし,25


In [29]:
soup_url = "../氷川きよし/2019-09-20/ticket-data1.html"
sc2 = JanniesScraper(datestr="2019-09-20",soup_url=soup_url,artname="氷川きよし")
sc2.store_data()
sc2.df["date"] = sc1.df["date"] - datetime.timedelta(days=60)
sc2.df["collected_at"] = sc1.df["collected_at"] - datetime.timedelta(days=60)
sc2.df

,name,date,collected_at,left_days,is_jannies,available,price_per_deal,unit,price_per_unit,after,after_and_js_jannies,artistname,age
0,JASRAC創立80周年記念 こころの歌人たちスペシャル 歌よ未来へ,2019-05-06,2019-04-23,4,1,1,9000,2,4500.0,1,1,氷川きよし,25
1,JASRAC創立80周年記念 こころの歌人たちスペシャル 歌よ未来へ,2019-05-06,2019-04-23,4,1,0,9000,2,4500.0,1,1,氷川きよし,25
2,JASRAC創立80周年記念 こころの歌人たちスペシャル 歌よ未来へ,2019-05-06,2019-04-23,4,1,1,17000,1,17000.0,1,1,氷川きよし,25
3,氷川きよしコンサートツアー2019 20周年大感謝祭！！,2019-05-06,2019-04-23,9,1,1,9000,2,4500.0,1,1,氷川きよし,25
4,氷川きよしコンサートツアー2019 20周年大感謝祭！！,2019-05-06,2019-04-23,9,1,1,15000,1,15000.0,1,1,氷川きよし,25
5,氷川きよしコンサートツアー2019 20周年大感謝祭！！,2019-05-06,2019-04-23,9,1,1,13000,1,13000.0,1,1,氷川きよし,25
6,氷川きよしコンサートツアー2019 20周年大感謝祭！！,2019-05-06,2019-04-23,9,1,1,14500,2,7250.0,1,1,氷川きよし,25
7,氷川きよしコンサートツアー2019 20周年大感謝祭！！,2019-05-06,2019-04-23,9,1,1,17000,1,17000.0,1,1,氷川きよし,25
8,氷川きよしコンサートツアー2019 20周年大感謝祭！！,2019-05-06,2019-04-23,9,1,0,23000,1,23000.0,1,1,氷川きよし,25
9,氷川きよしコンサートツアー2019 20周年大感謝祭！！,NaT,NaT,13,1,1,13000,2,6500.0,1,1,氷川きよし,25


In [30]:
soup_url = "../坂本冬美/2019-09-20/ticket-data1.html"
sc3 = JanniesScraper(datestr="2019-09-20",soup_url=soup_url,artname="坂本冬美")
sc3.store_data()
sc3.df["date"] = sc1.df["date"] - datetime.timedelta(days=70)
sc3.df["collected_at"] = sc1.df["collected_at"] - datetime.timedelta(days=70)
sc3.df

,name,date,collected_at,left_days,is_jannies,available,price_per_deal,unit,price_per_unit,after,after_and_js_jannies,artistname,age
0,JASRAC創立80周年記念 こころの歌人たちスペシャル 歌よ未来へ,2019-04-26,2019-04-13,4,1,1,9000,2,4500.0,1,1,坂本冬美,25
1,JASRAC創立80周年記念 こころの歌人たちスペシャル 歌よ未来へ,2019-04-26,2019-04-13,4,1,0,9000,2,4500.0,1,1,坂本冬美,25
2,JASRAC創立80周年記念 こころの歌人たちスペシャル 歌よ未来へ,2019-04-26,2019-04-13,4,1,1,17000,1,17000.0,1,1,坂本冬美,25
3,新・BS日本のうた,2019-04-26,2019-04-13,13,1,1,3200,1,3200.0,1,1,坂本冬美,25
4,新・BS日本のうた,2019-04-26,2019-04-13,13,1,1,3500,1,3500.0,1,1,坂本冬美,25
5,新・BS日本のうた,2019-04-26,2019-04-13,13,1,0,3000,1,3000.0,1,1,坂本冬美,25
6,新・BS日本のうた,2019-04-26,2019-04-13,13,1,1,3300,1,3300.0,1,1,坂本冬美,25
7,新・BS日本のうた,2019-04-26,2019-04-13,13,1,1,3980,1,3980.0,1,1,坂本冬美,25
8,五木ひろし 特別公演 歌・舞・奏スペシャル,2019-04-26,2019-04-13,18,1,1,8000,2,4000.0,1,1,坂本冬美,25
9,五木ひろし 特別公演 歌・舞・奏スペシャル,NaT,NaT,18,1,1,10000,2,5000.0,1,1,坂本冬美,25


In [31]:
df = pd.concat([sc1.df, sc2.df, sc3.df],ignore_index=True)
df["is_free_market"] = 1
df.to_csv("../分析対象データ群/ヤフオクとチケット流通センターの裁定取引/チケット流通センター演歌.csv")

In [34]:
d = pd.read_csv("../CSV/演歌/演歌.csv",index_col=0)
d["is_free_market"] = 0
d = d[d["price_per_unit"] >= 1000]
d

,date,collected_at,left_days,is_jannies,available,price_per_deal,unit,price_per_unit,after,after_and_js_jannies,artistname,is_free_market
name,,,,,,,,,,,,
氷川きよし特別公演新歌舞伎座5/23 夜S席1階8~10列センターブロック1 ~2枚,2019-05-23,2019-05-17,6,0,1,12000,2,6000.0,0,0,演歌,0
★2019年7月13日(土曜日)に大阪である大阪新歌舞伎座開場60周年記念市川由紀乃コンサートのチケット送料込み!,2019-07-13,2019-05-17,57,0,1,21000,1,21000.0,0,0,演歌,0
"ごごウタ NHK みんなの広場 ふれあいホール 5/24 (金) 1枚2枚様 入場整理券 小堺一機,塚原愛,おかゆ,門倉有希,北島兄弟,神野美伽,ほか",2019-05-24,2019-05-17,7,0,1,1300,1,1300.0,0,0,演歌,0
松平健川中美幸特別公演6月8日土曜日11時開演新歌舞伎座1階18列2枚ペア,2019-06-08,2019-05-17,22,0,1,5000,2,2500.0,0,0,演歌,0
氷川きよし 特別公演 新歌舞伎座,2019-05-19,2019-05-17,2,0,1,16000,1,16000.0,0,0,演歌,0
氷川きよし特別公演♪新歌舞伎座5/26(日)昼1階1列送料込み1枚,2019-05-26,2019-05-17,9,0,1,16000,1,16000.0,0,0,演歌,0
氷川きよし 特別公演,2019-05-17,2019-05-17,0,0,1,16000,1,16000.0,0,0,演歌,0
入場券「八代亜紀コンサート2019」ゲスト北川大介2019年6月14日夜の部津田沼文化ホール,2019-06-14,2019-05-17,28,0,1,1100,1,1100.0,0,0,演歌,0
森昌子祝還暦コンサート7月22日14時江戸川区総合文化センター。プレミアシート。7列20,2019-07-22,2019-05-17,66,0,1,15000,1,15000.0,0,0,演歌,0


In [37]:
df["is_jannies"] = 0
df["after_and_js_jannies"] = df["is_jannies"] * df["after"]
pd.concat([df,d],ignore_index=True).to_csv("../分析対象データ群/ヤフオクとチケット流通センターの裁定取引/チケット流通センターとヤフオク結合演歌.csv")

d = pd.read_csv("../分析対象データ群/ヤフオクとチケット流通センターの裁定取引/チケット流通センターとヤフオク結合演歌.csv",index_col=0)
d

/Users/aki/ticket_tenbai/my_env/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


,after,after_and_js_jannies,age,artistname,available,collected_at,date,is_free_market,is_jannies,left_days,name,price_per_deal,price_per_unit,unit
0,1,0,25.0,細川たかし,1,2019-06-22 00:00:00,2019-07-05 00:00:00,1,0,13,新・BS日本のうた,3200,3200.0,1
1,1,0,25.0,細川たかし,1,2019-06-22 00:00:00,2019-07-05 00:00:00,1,0,13,新・BS日本のうた,3500,3500.0,1
2,1,0,25.0,細川たかし,0,2019-06-22 00:00:00,2019-07-05 00:00:00,1,0,13,新・BS日本のうた,3000,3000.0,1
3,1,0,25.0,細川たかし,1,2019-06-22 00:00:00,2019-07-05 00:00:00,1,0,13,新・BS日本のうた,3300,3300.0,1
4,1,0,25.0,細川たかし,1,2019-06-22 00:00:00,2019-07-05 00:00:00,1,0,13,新・BS日本のうた,3980,3980.0,1
5,1,0,25.0,細川たかし,0,2019-06-22 00:00:00,2019-07-05 00:00:00,1,0,13,新・BS日本のうた,5000,5000.0,1
6,1,0,25.0,細川たかし,0,2019-06-22 00:00:00,2019-07-05 00:00:00,1,0,13,新・BS日本のうた,2800,2800.0,1
7,1,0,25.0,細川たかし,0,2019-06-22 00:00:00,2019-07-05 00:00:00,1,0,13,新・BS日本のうた,2800,2800.0,1
8,1,0,25.0,細川たかし,0,2019-06-22 00:00:00,2019-07-05 00:00:00,1,0,13,新・BS日本のうた,3000,3000.0,1
9,1,0,25.0,氷川きよし,1,2019-04-23 00:00:00,2019-05-06 00:00:00,1,0,4,JASRAC創立80周年記念 こころの歌人たちスペシャル 歌よ未来へ,9000,4500.0,2


,after,after_and_js_jannies,age,artistname,available,collected_at,date,is_free_market,is_jannies,left_days,name,price_per_deal,price_per_unit,unit
0,1,1,25.0,細川たかし,1,2019-06-22 00:00:00,2019-07-05 00:00:00,1,1,13,新・BS日本のうた,3200,3200.0,1
1,1,1,25.0,細川たかし,1,2019-06-22 00:00:00,2019-07-05 00:00:00,1,1,13,新・BS日本のうた,3500,3500.0,1
2,1,1,25.0,細川たかし,0,2019-06-22 00:00:00,2019-07-05 00:00:00,1,1,13,新・BS日本のうた,3000,3000.0,1
3,1,1,25.0,細川たかし,1,2019-06-22 00:00:00,2019-07-05 00:00:00,1,1,13,新・BS日本のうた,3300,3300.0,1
4,1,1,25.0,細川たかし,1,2019-06-22 00:00:00,2019-07-05 00:00:00,1,1,13,新・BS日本のうた,3980,3980.0,1
5,1,1,25.0,細川たかし,0,2019-06-22 00:00:00,2019-07-05 00:00:00,1,1,13,新・BS日本のうた,5000,5000.0,1
6,1,1,25.0,細川たかし,0,2019-06-22 00:00:00,2019-07-05 00:00:00,1,1,13,新・BS日本のうた,2800,2800.0,1
7,1,1,25.0,細川たかし,0,2019-06-22 00:00:00,2019-07-05 00:00:00,1,1,13,新・BS日本のうた,2800,2800.0,1
8,1,1,25.0,細川たかし,0,2019-06-22 00:00:00,2019-07-05 00:00:00,1,1,13,新・BS日本のうた,3000,3000.0,1
9,1,1,25.0,氷川きよし,1,2019-04-23 00:00:00,2019-05-06 00:00:00,1,1,4,JASRAC創立80周年記念 こころの歌人たちスペシャル 歌よ未来へ,9000,4500.0,2
